In [1]:
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding
import tensorflow as tf
import numpy as np
import random

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir("/content/gdrive/My Drive/")


# 数据预处理

加载文本获取jk格裙的名字


In [4]:
data=open('./nlp/jk.txt').read()
data=data.lower()
char=sorted(set(data))
char_num=len(char)
print(f'样本长度{len(data)},字符数量{char_num}')

样本长度405,字符数量221


创建对照列表。char2id表示字符映射到数字。id2char表示数字映射到字符.  
'\n'表示<EOS>，对应0.

In [5]:
char2id={i:u+1 for u,i in enumerate(char)}
id2char={u+1:i for u,i in enumerate(char)}
char2id,id2char

({'\n': 1,
  '/': 2,
  '1': 3,
  '5': 4,
  'c': 5,
  'm': 6,
  's': 7,
  '一': 8,
  '不': 9,
  '中': 10,
  '主': 11,
  '之': 12,
  '乌': 13,
  '也': 14,
  '书': 15,
  '事': 16,
  '云': 17,
  '代': 18,
  '伊': 19,
  '伦': 20,
  '信': 21,
  '偏': 22,
  '傲': 23,
  '光': 24,
  '公': 25,
  '兽': 26,
  '内': 27,
  '冬': 28,
  '冰': 29,
  '凛': 30,
  '刀': 31,
  '化': 32,
  '十': 33,
  '千': 34,
  '华': 35,
  '南': 36,
  '取': 37,
  '吉': 38,
  '名': 39,
  '吟': 40,
  '吹': 41,
  '咩': 42,
  '啊': 43,
  '啵': 44,
  '囚': 45,
  '园': 46,
  '坞': 47,
  '多': 48,
  '夜': 49,
  '大': 50,
  '天': 51,
  '奈': 52,
  '女': 53,
  '奶': 54,
  '姐': 55,
  '姬': 56,
  '威': 57,
  '婪': 58,
  '子': 59,
  '学': 60,
  '宇': 61,
  '安': 62,
  '宙': 63,
  '宴': 64,
  '小': 65,
  '少': 66,
  '山': 67,
  '岁': 68,
  '岛': 69,
  '川': 70,
  '帽': 71,
  '干': 72,
  '律': 73,
  '心': 74,
  '性': 75,
  '慢': 76,
  '戏': 77,
  '户': 78,
  '托': 79,
  '抹': 80,
  '拂': 81,
  '敦': 82,
  '数': 83,
  '斜': 84,
  '日': 85,
  '昏': 86,
  '星': 87,
  '春': 88,
  '昨': 89,
  '晖': 90,
  '晨': 91,
  '暮': 

创建训练集

In [7]:
with open('./nlp/jk.txt') as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]
maxlen=max([len(i) for i in examples ])
examples[0]

'白夜霓虹'

In [38]:
maxlen

5

将训练集的字符变为数字编码

In [8]:
X,Y=[],[]
for index in range(len(examples)):#对于examples（也就是由名字组成的列表）的下标进行遍历
    x =[char2id[ch] for ch in examples[index]]#对于该下标对应的名字，如index=0对应的examples0是白夜霓虹中的每个字符，找到它的数字id，并且存放在x列表中
    y =x[1:]+[char2id["\n"]]#将从第1个字符开始往后的字符后统统加上[1]，补齐
    X.append(x)
    Y.append(y)
X[0],Y[0]

([152, 49, 208, 186], [49, 208, 186, 1])

将输出padding为同一长度

In [9]:
X=np.array(X)
Y=np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [10]:
padded_X=tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=maxlen,padding='post',value=0)#这里按照最大长度补了0

In [11]:
padded_Y=tf.keras.preprocessing.sequence.pad_sequences(Y,maxlen=maxlen,padding='post',value=0)

In [12]:
print(padded_X.shape,padded_Y.shape)

(104, 5) (104, 5)


将训练集随机打乱

In [13]:
np.random.seed(3)
np.random.shuffle(X)
np.random.seed(3)
np.random.shuffle(Y)

In [14]:
X[3],Y[3]

([218, 86], [86, 1])

In [32]:
X

array([list([29, 120]), list([125, 175]), list([136, 157]),
       list([218, 86]), list([117, 101, 131, 138]), list([88, 85, 64]),
       list([207, 99]), list([121, 122]), list([89, 85, 88, 90]),
       list([177, 179, 198, 198]), list([171, 168]), list([92, 117, 40]),
       list([61, 63]), list([66, 53, 74, 16]), list([65, 218, 217]),
       list([65, 110]), list([50, 65, 55]), list([125, 47, 214, 105]),
       list([216, 12, 193]), list([162, 184]), list([162, 140]),
       list([81, 188]), list([103, 113, 125, 154]), list([174, 176]),
       list([195, 58]), list([194, 123]), list([17, 97]), list([92, 24]),
       list([213, 77]), list([163, 45, 216]), list([152, 133]),
       list([65, 163, 71]), list([94, 219, 211, 214]), list([171, 168]),
       list([209, 216, 10]), list([173, 173, 107, 107]),
       list([20, 82, 15, 21]), list([182, 181, 118, 116]), list([42, 43]),
       list([150, 158, 66, 53]), list([143, 192, 26]), list([146, 109]),
       list([190, 149, 44, 44]), list

In [33]:
Y

array([list([120, 1]), list([175, 1]), list([157, 1]), list([86, 1]),
       list([101, 131, 138, 1]), list([85, 64, 1]), list([99, 1]),
       list([122, 1]), list([85, 88, 90, 1]), list([179, 198, 198, 1]),
       list([168, 1]), list([117, 40, 1]), list([63, 1]),
       list([53, 74, 16, 1]), list([218, 217, 1]), list([110, 1]),
       list([65, 55, 1]), list([47, 214, 105, 1]), list([12, 193, 1]),
       list([184, 1]), list([140, 1]), list([188, 1]),
       list([113, 125, 154, 1]), list([176, 1]), list([58, 1]),
       list([123, 1]), list([97, 1]), list([24, 1]), list([77, 1]),
       list([45, 216, 1]), list([133, 1]), list([163, 71, 1]),
       list([219, 211, 214, 1]), list([168, 1]), list([216, 10, 1]),
       list([173, 107, 107, 1]), list([82, 15, 21, 1]),
       list([181, 118, 116, 1]), list([43, 1]), list([158, 66, 53, 1]),
       list([192, 26, 1]), list([109, 1]), list([149, 44, 44, 1]),
       list([93, 1]), list([100, 98, 21, 1]), list([5, 6, 2, 7, 1]),
       list(

In [15]:
print(type(padded_X[0]))

<class 'numpy.ndarray'>


In [16]:
train_db = tf.data.Dataset.from_tensor_slices((padded_X, padded_Y))#加载一下这两个数据集
train_db=train_db.batch(32,drop_remainder=True)#分为32一组

In [70]:
padded_X[0]

array([152,  49, 208, 186,   0], dtype=int32)

In [37]:
padded_Y[0]

array([ 49, 208, 186,   1,   0], dtype=int32)

In [17]:
train_iter = iter(train_db)
# next() 返回迭代器的下一个项目
sample = next(train_iter)
print('batch:', sample[0].shape, sample[1].shape)
print(sample[0][0],sample[1][0])

batch: (32, 5) (32, 5)
tf.Tensor([152  49 208 186   0], shape=(5,), dtype=int32) tf.Tensor([ 49 208 186   1   0], shape=(5,), dtype=int32)


******************

----------------------------------------------------------------------

# 创建模型 

创建模型，注意embdding层是vocab_size+1，应为加入了padding 0.
最后的softmax 也是vocab_size+1
注意是return_sequences=True,应为每一个时刻我们都要产生输出
![](pic/dis1.png)

In [18]:
class My_model(tf.keras.Model):
    def __init__(self,vocab_size,rnn_units):#要传入字符长度和rnn的单元数
        super(My_model,self).__init__()
        self.embedding=Embedding(vocab_size+1,5,name='emb')#第一层有vocab长度的层作为嵌入层
        self.rnn=SimpleRNN(rnn_units,return_sequences=True,name='rnn')#第二层有
#         self.d1=Dense(64,activation='relu',name='d1')
        self.d2=Dense(vocab_size+1,activation='softmax',name='d2')#输出也要是比词汇表长1,也就是222
    
    def call(self,x):
        x=self.embedding(x)#首先将x编码层
        x=self.rnn(x)#然后rnn进入
#         x=self.d1(x)
        x=self.d2(x)#最后输出x也和输入一样的大小
        return x
        

In [39]:
model=My_model(char_num,16)#vocab_size

## 取样
![](pic/dis3.png)

我们在一个时刻会得到一个预测，然后我们需要将这个预测结果作为下一个时间点的输入，然后进行下一次预测。
我们输出的yt是softmax之后的结果，代表我们预测下一个词的概率，然后我们需要依照概率进行抽样（注意不像往常依照取argmax，因为这样我们很有可能产生死循环）。

In [67]:
import random

def sample(model):
    seed=0
    name=[]
    for i in range(8):#这里就是第一个字取几个
        a=[random.randint(1,221)]#返回任意一个，221是字符的个数
        b=tf.expand_dims(a,0)#增加1维
        ans=[id2char[a[0]].upper()]#找到原字典里的这个字并且放在列表里
        for i in range(1):#除了第一个字以外，还要接几个字
            pred=model(b)
            pred=tf.squeeze(pred)#删除张量
            pred=np.array(pred)#转换格式
            #如果要固定除了第一个字以外的后面的字，那么这里设置一下随机数种子
            #np.random.seed(i+seed) 
            idx = np.random.choice(list(range(222)), p=pred.ravel())#将多维数组转换为一维数组，p是取list中各个元素的概率，只得出一个随机数
            if idx==0 or idx==1:
                break
            next_word=id2char[idx]#把这个下标的字赋值出来
            ans.append(next_word)#添加到ans表中
            a=[char2id[next_word]]#这句可以不用吧
            b=tf.expand_dims(a,0)
            seed+=1
        
        ans=''.join(ans)#将ans列表打平成字符串
        name.append(ans)#添加到名字里
    for n in name:
        if n is not None:
            print(n)

## 定义优化器和损失
我们要将padding 0 位置上产生的损失mask掉

In [21]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam(1e-3)
def loss_function(y_true,y_pred):
    # 我们将0mask掉，不计算0的损失
    mask=tf.math.logical_not(tf.math.equal(y_true,0))
    loss=loss_object(y_true,y_pred)
    mask=tf.cast(mask,dtype=loss.dtype)
    loss*=mask
    return tf.reduce_mean(loss)

## 进行训练
训练的时候，我们不像预测进行采样，因为训练的时候，我们预测的结果很有可能是错的，然后我们传入错误的结果进行预测，那么产生的下一个结果就更加糟糕了。

所以我们使用教师强制（teaching force）的方式。将下一个正确的答案输入到模型，然后进行下一次的预测。
此外，我们需要对模型进行梯度裁剪，避免梯度爆炸。
![](pic/dis2.png)

In [22]:
@tf.function
def train_step(inp,targ):#返回的是每个batch的loss
    loss=0
    with tf.GradientTape() as tape:
        # 教师强制-将目标词作为下一个输入
        model_input=inp[:,0]#取出这32条数据的第一个字
        model_input=tf.expand_dims(model_input,1)#变成张量
        for t in range(1,targ.shape[1]):
            # 将编码器输出传到解码器
            predictions=model(model_input)#得到预测结果
            loss+=loss_function(targ[:,t],predictions)#和教师的后面几个列依次进行比较
            # 使用教师强制
            model_input=tf.expand_dims(targ[:,t],1)#然后把target变成张量，变成输入
        batch_loss=(loss/int(targ.shape[1]))
        variables=model.variables#取出变量
        # 对每一个变量计算梯度
        gradients=tape.gradient(loss,variables)
#         print(type(gradients))
#         print(gradients[0])
#         print(gradients[1])
        gradients, _ = tf.clip_by_global_norm(gradients,3)#所有梯度的平方和，梯度裁剪
#         gradients = [tf.clip_by_value(gards, -3, 3) for gards in gradients if gards is not None]
        # Apply gradients to variables
        optimizer.apply_gradients(zip(gradients,variables))
        return batch_loss
            

In [68]:
EPOCHS=100
steps_per_epoch=len(X)//32
for epoch in range(EPOCHS):
    
    total_loss=0
    
    for (batch,(inp,targ)) in enumerate(train_db.take(steps_per_epoch)):
        batch_loss=train_step(inp,targ)
        total_loss+=batch_loss
     
        # 每 2 个周期（epoch），保存（检查点）一次模型
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,total_loss ))
    print()
    # 在每一次训练周期进行输出，可以查看一开始生成的名字乱七八糟，后来的名字逐渐有规律了
    sample(model)
    print()


Epoch 1 Loss 0.5581

藏芝
多杉
铜粉
豆鸭
托沙
莓楂
暮书
春子

Epoch 2 Loss 0.5570

诗椿
取电
铭荷
慢斜
电也
柔宴
取野
见坞

Epoch 3 Loss 0.5558

溟林
饼雀
温c
化溟
柠面
姐天
莓泷
缥野

Epoch 4 Loss 0.5547

良冰
见水
淋绯
满璃
柔柠
楂囚
春梦
燕敦

Epoch 5 Loss 0.5535

云校
之拂
电虹
枫肝
奶岛
澄淋
满宇
代林

Epoch 6 Loss 0.5524

取户
河王
C敦
学枫
奶竹
高晨
衣野
伊名

Epoch 7 Loss 0.5513

雀纱
柔曦
葡月
多烟
灰水
宴暮
宴
泷伦

Epoch 8 Loss 0.5501

芝空
醉水
枫盐
胡饼
杉留
数主
事岁
葡渔

Epoch 9 Loss 0.5490

刀奈
凛高
肉青
内衣
律树
死虹
主冰
淋岁

Epoch 10 Loss 0.5479

花柔
深纱
竞桃
莓凛
葡慢
步干
托少
山南

Epoch 11 Loss 0.5468

面暮
女松
校良
温茶
偏死
数柔
川良
芝酒

Epoch 12 Loss 0.5457

来瓜
铜草
公霓
园不
苔豆
心宇
烟校
十茶

Epoch 13 Loss 0.5445

伦竹
星化
子玫
抹书
铜托
草莫
月月
豆溟

Epoch 14 Loss 0.5434

见糖
水葡
姬缨
多
野曦
鱼柔
冬之
鱼河

Epoch 15 Loss 0.5423

碳满
安薄
坞青
木冬
晨天
汽玫
死冰
干夜

Epoch 16 Loss 0.5413

葡淋
藏取
死芝
樱西
信s
花汽
水茶
岛鸟

Epoch 17 Loss 0.5402

线学
林取
华豆
高野
柔啊
日杉
书竹
海行

Epoch 18 Loss 0.5391

姬取
河玻
糖莓
漫泷
也南
醉s
学安
西莓

Epoch 19 Loss 0.5380

猪不
胡
缥抹
糖也
晖十
梦c
昨水
园盐

Epoch 20 Loss 0.5369

良月
斜冰
数律
也渔
柠霓
内黛
深女
之/

Epoch 21 Loss 0.5359

阳薯
水阳
面枫
阳胡
竹角
化漫
傲诗
璃c

Epoch 22 Loss 0.5348

杀学
